In [1]:
using BenchmarkTools
using Dates

In [2]:
include("../src/foldexity.jl")
include("../src/entropy.jl")

In [3]:
fxpdb("../testpdb/AF-Q8VCK6-F1-model_v4.pdb", 3, 0.5) 


Starting foldexity...


(4.505749377134266, 1.3746020152515541, 86, 0.2621951219512195, 328, [0.0 0.3617100119590759 … 2.1261956691741943 2.1296846866607666; 0.3617100119590759 0.0 … 2.1606688499450684 2.2002992630004883; … ; 2.1261956691741943 2.1606688499450684 … 0.0 1.0081433057785034; 2.1296846866607666 2.2002992630004883 … 1.0081433057785034 0.0])

In [236]:
fxdir("../testpdb", "fxdata.tsv", 4, 1.0)
rm("fxdata.tsv")

5 files are collected 
Starting foldexity...


20.0%

┣████████▋                                  ┫ 1/5 [00:00<Inf:Inf, InfGs/it]
60.0%┣████████████████████████████▉                   ┫ 3/5 [00:01<00:01, 2it/s]
80.0%┣██████████████████████████████████████▍         ┫ 4/5 [00:02<00:01, 2it/s]
100.0%┣███████████████████████████████████████████████┫ 5/5 [00:02<00:00, 3it/s]


In [35]:
pdb = readpdb_backbone("../testpdb/AF-Q8VCK6-F1-model_v4.pdb")
writepdb(pdb, "../testpdb/backbone.pdb")
fsize = 8

xyzcoords = pdb2xyz(pdb)
xyzcoords = matrix2fragments(xyzcoords, fsize)

pdbmatrix = pdb2pdbmatrix(pdb)
pdbfragments = matrix2fragments(pdbmatrix, fsize)

fxity, aver_rmsd, nclusts, norm_nclusts, nfrags, matrix = fxity_kabsh(xyzcoords)

(7.309983070517202, 8.600171516053804, 220, 0.6811145510835913, 323, [0.0 12.18087387084961 … 20.524507522583008 20.470008850097656; 12.18087387084961 0.0 … 22.167490005493164 22.168933868408203; … ; 20.524507522583008 22.167490005493164 … 0.0 4.060173988342285; 20.470008850097656 22.168933868408203 … 4.060173988342285 0.0])

In [36]:
# dump PDB fragments
pdb = readpdb_backbone("../testpdb/small.pdb")
pdbfragments = matrix2fragments(pdbmatrix, 20)
for (i, frag) in enumerate(pdbfragments)
    writepdb(pdbmatrix2pdb(frag), "../frags/frag$i.pdb")
end

In [37]:
pfrag1 = xyzcoords[10]
pfrag2 = xyzcoords[11]

kabsch_umeyama(pfrag1, pfrag2)

0.36361846f0

In [123]:
pdb = readpdb_calpha("../testpdb/small.pdb")
xyz = pdb2xyz(pdb)
print()

In [240]:
import Pkg; Pkg.add("NearestNeighbors")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
    Updating `/spin1/home/linux/saakyanh2/.julia/environments/v1.11/Project.toml`
  [b8a86587] + NearestNeighbors v0.4.21
  No Changes to `/spin1/home/linux/saakyanh2/.julia/environments/v1.11/Manifest.toml`


In [241]:
using NearestNeighbors

In [256]:
data = xyz'

# Create trees
kdtree = KDTree(data; leafsize = 25)
balltree = BallTree(data, Minkowski(3.5); reorder = false)
brutetree = BruteTree(data)


BruteTree{StaticArraysCore.SVector{3, Float32}, Euclidean}
  Number of points: 50
  Dimensions: 3
  Metric: Euclidean(0.0)
  Reordered: false

In [257]:
k = 5
point = rand(3)

kdtree = KDTree(data)
idxs, dists = knn(kdtree, point, k, true)



([50, 48, 47, 49, 25], Float32[10.844793, 10.958652, 11.385086, 12.879178, 13.850678])